## Logistic Regression

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

- Binary cross-entropy
$$ loss = -\frac{1}{m}\sum y \log(H(x))+(1-y)(\log(1-H(x))   $$

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [3]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
W = torch.zeros([2,1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [5]:
z = torch.matmul(x_train, W) + b
pred = 1/(1+torch.exp(-z))  # or torch.sigmoid()

In [6]:
pred

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)

In [7]:
loss_sum = -(y_train*torch.log(pred) + (1-y_train)*torch.log(1-pred))
print(loss_sum)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [8]:
loss = loss_sum.mean()

In [9]:
optimizer = optim.SGD([W,b], lr=0.5)

In [10]:
for step in range(200):
    z = torch.matmul(x_train, W) + b
    pred = 1/(1+torch.exp(-z))  # or torch.sigmoid()
    loss_sum = -(y_train*torch.log(pred) + (1-y_train)*torch.log(1-pred))  # or F.binary_cross_entropy()
    loss = loss_sum.mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step%20==0:
        print('step:',step, 'loss:', loss.item())

step: 0 loss: 0.6931471824645996
step: 20 loss: 0.41267549991607666
step: 40 loss: 0.3487008512020111
step: 60 loss: 0.3005836308002472
step: 80 loss: 0.26288607716560364
step: 100 loss: 0.23294083774089813
step: 120 loss: 0.2087985724210739
step: 140 loss: 0.1890368014574051
step: 160 loss: 0.17262451350688934
step: 180 loss: 0.15881042182445526


### Accuracy evaluation

In [11]:
z = torch.matmul(x_train, W) + b
pred = 1/(1+torch.exp(-z))

In [12]:
pred

tensor([[0.0297],
        [0.1575],
        [0.2999],
        [0.7836],
        [0.9410],
        [0.9806]], grad_fn=<MulBackward0>)

- pred >= 0.5면 label 1이라 판단

In [13]:
pred_label = (pred >= 0.5)
print(pred_label)

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1]], dtype=torch.uint8)


In [14]:
accuracy = (pred_label.float()==y_train)
print(accuracy)

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.uint8)


In [15]:
accuracy.float().mean().item()

1.0

## Logistic Regression with nn.Module

In [16]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1, bias=True)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [17]:
model = LogisticRegression()

In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [19]:
for step in range(200):
    pred = model(x_train)
    loss = F.binary_cross_entropy(pred, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step%20==0:
        print('step:',step, 'loss:', loss.item())

step: 0 loss: 0.6457765698432922
step: 20 loss: 0.42508742213249207
step: 40 loss: 0.35937222838401794
step: 60 loss: 0.308942586183548
step: 80 loss: 0.26947149634361267
step: 100 loss: 0.23820219933986664
step: 120 loss: 0.21306723356246948
step: 140 loss: 0.19255273044109344
step: 160 loss: 0.17556141316890717
step: 180 loss: 0.16129539906978607
